# OpenStreetMapX

Biblioteka [OpenStreetMapX](https://github.com/pszufe/OpenStreetMapX.jl) pozwala  pracować na plikach <tt>.osm</tt> za pomocą języka Julia. Jest w całości napisana w Julii, jej kluczowe funkcje można podzielić na podstawowe kategorie:

- praca z danymi przestrzennymi;
- import i parsowanie plików mapowych;
- tworzenie grafu reprezentującego sieć drogową;
- algorytmy wyznaczania tras.

In [2]:
using OpenStreetMapX

## 1. Praca z danymi przestrzennymi

Podstawową funkcją na jaką pozwala biblioteka [OpenStreetMapX](https://github.com/pszufe/OpenStreetMapX.jl) jest praca na danych przestrzennych. W ramach biblioteki [zaimplementowane zostały 3 podstawowe systemy współrzędnych geograficznych:](https://en.wikipedia.org/wiki/Geographic_coordinate_system)
- <b>Latitude-Longitude-Altitude</b> (LLA)
- <b>Earth-centered, Earth-fixed (ECEF)</b> [![ECEF](https://upload.wikimedia.org/wikipedia/commons/8/88/Ecef.png "ECEF")](https://en.wikipedia.org/wiki/ECEF)
- <b>East, North, Up</b> (ENU) [![ENU](https://upload.wikimedia.org/wikipedia/commons/thumb/7/73/ECEF_ENU_Longitude_Latitude_relationships.svg/800px-ECEF_ENU_Longitude_Latitude_relationships.svg.png "ENU")](https://en.wikipedia.org/wiki/Local_tangent_plane_coordinates)


Wszystkie systemy zostały zaiplementowane jako [oddzielne typy danych](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/types.jl), wraz z konstruktorem <tt>XYZ</tt>, dzięki któremu ułatwia tworzenie i modyfikowanie koordynat:

In [7]:
SGH = OpenStreetMapX.XYZ(52.208842, 21.008560, 0.0)

OpenStreetMapX.XYZ(52.208842, 21.00856, 0.0)

In [8]:
SGH_LLA = OpenStreetMapX.LLA(SGH)

LLA(21.00856, 52.208842, 0.0)

Oczywiście możliwa jest też [konwersja pomiędzy różnymi systemami;](https://en.wikipedia.org/wiki/Geographic_coordinate_conversion) w tym celu konieczne jest wykorzystanie dwóch kolejnych pojęć zdefiniowanych w ramach biblioteki:
- elipsoidy ([biblioteka pozwala na zdefiniowanie własnej](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/types.jl#L101) lub [wykorzystanie jednej ze standardowych](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/points.jl))
- [granic obszaru i punktu referencyjnego](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/bounds.jl) w przypadku współrzędnych lokalnych (ENU).

[Konwersja](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/conversion.jl) jest prosta i intuicyjna - wystarczy wykorzystać odpowiednią funkcję (której nazwa odpowiada systemowi na który chcemy przekonwertować współrzędne):

In [9]:
SGH_ECEF = OpenStreetMapX.ECEF(SGH_LLA)

ECEF(3.6501926662423294e6, 4.707299855716553e6, 2.2722797836972033e6)

Ponadto możliwe jest konwertowanie całych kolekcji punktów, a nie tylko pojedynczych z nich.

## 2. Import i parsowanie plików mapowych

Aby lepiej zrozumieć sposób działania biblioteki rozpatrzmy prosty przykład. Wczytajmy plik z mapą Winnipeg w Kanadzie:

In [13]:
pth = "C:\\Users\\p\\Desktop\\learningsimdata";
name = "Winnipeg CMA.osm";

In [41]:
@time map = OpenStreetMapX.parseOSM(joinpath(pth,name));

 32.257581 seconds (113.43 M allocations: 7.912 GiB, 30.48% gc time)


Aby zapewnić sprawne działanie kodu, kluczowe jest działania [parsera](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/parseMap.jl#L60), który będzie w stanie w efektywny sposób wczytać i przygotować do pracy plik <tt>.osm</tt>. Opiera się on na lekkiej [implementacji](https://github.com/JuliaIO/LibExpat.jl) parsera plików <tt>XML</tt> [Expat](https://libexpat.github.io/) i tworzy obiekt typu [<tt>OSMData</tt>](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/types.jl#L185), który przechowuje cztery podstawowe typy prymitywne:
- Węzły (<b>nodes</b>) – pojedyncze punkty określone za pomocą ich współrzędnych geograficznych.
- Linie (<b>ways</b>) – uporządkowane listy węzłów służące do opisania polilinii (linii łamanych) i wielokątów (np. dróg, budynków).
- Relacje (<b>relations</b>) – uporządkowane listy zawierające w sobie zarówno węzły, linie jak i inne relacja (nazywane członkami (members)), które dodatkowo mogą mieć przypisaną rolę (role). Służą do opisywania złożonych elementów przestrzeni, np. autostrad czy też linii autobusowych.
- Tagi (<b>tags</b>) – para klucz-wartość służąca do przechowywania metadanych (charakterystyk – features) powyższych elementów mapy. Posiada określony sposób kodowania. 


In [17]:
fieldnames(OpenStreetMapX.OSMData)

(:nodes, :ways, :relations, :features, :bounds, :way_tags, :relation_tags)

In [43]:
map.bounds;

In [24]:
map.nodes;

In [26]:
map.features;

In [30]:
map.ways;

In [33]:
map.way_tags;

In [34]:
map.relations;

In [36]:
map.relation_tags;

Biblioteka umożliwia też [przycinanie wczytanych danych](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/crop.jl) do odpowiedniego rozmiaru przy wykorzystaniu podanych powyżej granic:

In [37]:
map = OpenStreetMapX.crop!(map)

Typy prymitiwne są zdefiniowane w ogólny sposób, aby możliwa była dalsza na nich praca konieczne jest stworzenie odpowiednich narzędzi dzięki którym możliwe będzie ich [filtrowanie (oddzielenie dróg od budynków, etc.) i klasyfikowanie](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/classification.jl):

In [49]:
roads = OpenStreetMapX.extract_highways(map.ways); #drogi wewnątrz biblioteki

In [48]:
buildings = OpenStreetMapX.extract_buildings(map.ways); #budynki

Klasyfikacja i filtrowanie odbywają się za pomocą odpowiednich zdefiniowanych [kategorii:](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/classes.jl)

In [63]:
roadways = OpenStreetMapX.filter_roadways(roads); #drogi dla samochodów

In [64]:
highways = OpenStreetMapX.filter_roadways(roads,levels =  Set(1)); #jedynie autostrady

In [62]:
roadways_classes = OpenStreetMapX.classify_roadways(roadways); #mapowanie dróg na kategorie

Możliwe jest też wyciąganie kluczowych [charakterystyk](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/intersections.jl#L7) [dróg](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/classification.jl#L5) z ich tagów:

In [66]:
OpenStreetMapX.getlanes(highways[4])

2

In [67]:
OpenStreetMapX.oneway(highways[4])

true

Biblioteka pozwala także na pracę na węzłach pliku mapowego. Na przykład [dodawanie nowych](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/nodes.jl#L5):

In [74]:
OpenStreetMapX.add_new_node!(map.nodes, LLA(49.152,-96))

5279456212340186414

Wyszukiwanie [najbliższego węzła](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/nodes.jl#L20):

In [80]:
point = OpenStreetMapX.ENU(LLA(49.153,-98.39), center(map.bounds));
nodes = OpenStreetMapX.ENU(map.nodes, center(map.bounds));

In [86]:
OpenStreetMapX.nearest_node(nodes, point);

Węzłów w [określonym promieniu](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/nodes.jl#L63):

In [85]:
OpenStreetMapX.nodes_within_range(nodes, point, 500.0);

a także [centroidów](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/nodes.jl#L100).

## 3. Tworzenie grafu reprezentującego sieć drogową


Biblioteka umożliwia stworzenie skierowanego grafu, który reprezentuje sieć drogową dowolnego pliku <tt>osm</tt>. Graf zbudowany jest za pomocą biblioteki [<tt>LightGraphs.jl](https://github.com/JuliaGraphs/LightGraphs.jl), dzięki czemu możliwe jest wykorzystanie zaimplementowanych w niej metod. Sieć drogową reprezentuje obiekt typu [<tt>MapData</tt>](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/types.jl#L228):

In [ ]:
struct MapData
    bounds::Bounds{LLA}
    nodes::Dict{Int,ENU}
    roadways::Array{Way,1}
    intersections::Dict{Int,Set{Int}}
    # Transporation network graph data and helpers to increase routing speed
    g::LightGraphs.SimpleGraphs.SimpleDiGraph{Int64} # Graph object
    v::Dict{Int,Int}                             # (node id) => (graph vertex)
    e::Array{Tuple{Int64,Int64},1}                # Edges in graph, stored as a tuple (source,destination)
    w::SparseArrays.SparseMatrixCSC{Float64, Int}   # Edge weights, indexed by graph id
    class::Vector{Int}                           # Road class of each edge
	#MapData(bounds, nodes, roadways, intersections) = new(bounds, nodes, roadways, intersections, LightGraphs.SimpleGraphs.SimpleDiGraph{Int64}(), Dict{Int,Int}(), Tuple{Int64,Int64}[],  SparseMatrixCSC(Matrix{Float64}(undef,0,0)),Int[])
end

Do tworzenia grafu wykorzystywane są dwa kolejne typy danych: [segmenty](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/types.jl#L174) (odcinki dróg od skrzyżowania do skrzyżowania) i [skrzyżowania](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/intersections.jl#L33) (zbiory odcinków, które przecinają się w danym węźle).

Graf tworzy funkcja [get_map_data](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/parseMap.jl#L86), która pozwala na wybór tego czy graf ma zawierać wszystkie węzły czy też tylko skrzyżowania dróg.

In [90]:
map_data =  OpenStreetMapX.get_map_data(pth,name,use_cache = false);

## 4. Algorytmy wyznaczania tras

Dla gotowego grafu możliwe jest wyznaczania tras pomiędzy punktami A i B. Można to robić dwojako:
- za pomocą algorytmów wyszukiwania najktórszej ścieżki zaimplementowanych w bibliotece
- za pomocą odpytania <b>[Google Directions API](https://developers.google.com/maps/documentation/directions/start)</b>

### 4.1. Algorytmy najkrószej ścieżki

Wyszukiwanie najkrótszej ścieżki jest zaimplentowane z wykorzystaniem zawartej w bibliotece <tt>LightGraphs.jl</tt> [implementacji algorytmu Dijkstry](https://github.com/JuliaGraphs/LightGraphs.jl/blob/master/src/shortestpaths/dijkstra.jl). Wybór ścieżki kontroluje funkcja [<tt>find_route</tt>](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/routing.jl#L139), która pozwala na wyznaczenie ścieżki dla dowolnej pary wierzchołków grafu przy wykorzystaniu dowolnej macierzy wag (definiowanej jako [macierz rzadka):](https://docs.julialang.org/en/v1.1/stdlib/SparseArrays/#Sparse-Arrays-1)

In [97]:
A = OpenStreetMapX.point_to_nodes(OpenStreetMapX.generate_point_in_bounds(map_data), map_data);
B = OpenStreetMapX.point_to_nodes(OpenStreetMapX.generate_point_in_bounds(map_data), map_data);

In [98]:
route = OpenStreetMapX.find_route(map_data, A,B,
    map_data.w, true, true)

4-element Array{Any,1}:
       [2110154909, 2110155355, 2110148981, 2110148095, 2110146531, 2110143918, 2110141846, 2110141472, 2110140990, 2110140378  …  571663906, 1998846174, 571663895, 339923734, 1998846204, 421392250, 1987878251, 1987878333, 1987878307, 1987878294]
 114743.29393174646                                                                                                                                                                                                                                   
 114743.29393174646                                                                                                                                                                                                                                   
   6602.120007085567                                                                                                                                                                                                                       

Zaimplementowane są także dwie formy tego algorytmu, które pozwalają na wyznaczenia <b>najkrótszej trasy</b> (wykorzystując do tego zdefiniowane w pliku <tt>MapData</tt> odległości pomiędzy węzłami) i <b>najszybszej trasy</b> (gdzie czasy przejazdu pomiędzy węzłami są wyznaczane za pomocą zdefiniowanych [ograniczeń prędkości na danym typie drogi](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/speeds.jl)).

In [99]:
shortest_route = OpenStreetMapX.shortest_route(map_data, A,B)

([2110154909, 2110155355, 2110148981, 2110148095, 2110146531, 2110143918, 2110141846, 2110141472, 2110140990, 2110140378  …  571663906, 1998846174, 571663895, 339923734, 1998846204, 421392250, 1987878251, 1987878333, 1987878307, 1987878294], 114743.29393174646, 6602.120007085567)

In [100]:
fastest_route = OpenStreetMapX.fastest_route(map_data, A,B)

([2110154909, 2110155355, 2110148981, 2110148095, 2110146531, 2110143918, 2110141846, 2110141472, 2110140990, 2110140378  …  421392250, 1998846188, 1987878339, 1987878283, 441073257, 441073252, 421392248, 3928833134, 1987878265, 1987878294], 121289.56985907798, 5195.834974360792)

Ponadto możliwe jest wyznaczenie trasy dla trzech punktów i wyszukanie takiego punktu pośredniego, który dla zadanych wag [minimalizuje trasę A-C-B](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/routing.jl#L239).

Biblioteka pozwala też na znalezienie wszystkich punktów dla których [czas przejazdu lub odległość jest mniejsza niż zadana wartość](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/routing.jl#L277):

In [103]:
OpenStreetMapX.nodes_within_driving_time(map_data,[A],600.0)

([2110161060, 2110167967, 2110169074, 2110165097, 2110109806, 2110116724, 2110177650, 2110195809, 2110160428, 2110163900  …  2110140731, 2110151288, 2110125241, 2110195812, 2110123296, 2110161203, 2110132311, 2110176017, 2110143475, 2110175112], [167.737, 355.681, 452.729, 273.309, 589.443, 493.913, 314.652, 552.519, 329.756, 254.326  …  274.618, 352.766, 396.414, 544.914, 434.038, 206.983, 509.949, 347.958, 214.97, 293.894])

### 4.2. Google Directions API

Algorytmy najkrótszej ścieżki nie są jedynym sposobem na wyszukiwanie trasy zaimplementowanym w bibliotece <tt>OpenStreetMapX</tt>. Możliwe jest też skorzystanie z [Google Directions API](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/google_routing.jl) (konieczne jest posiadanie odpowiedniego klucza):

In [104]:
googleapi = "C:\\Users\\p\\Desktop\\datasets\\googleapi.key";

In [106]:
google_route = get_google_route(A,B,
                            map_data,
                            googleapi)

UndefVarError: UndefVarError: warn not defined

Wyniki Google'a zwracane są w formacie pliku [JSON](), który jest odpowiednio parsowany i wyciągane są z niego informacje na temat węzłów przez które przechodzi trasa. Mapy Google’a są szczegółowe i przechowują relatywnie dużo informacji na temat otoczenia, przez co pliki są zakodowane w formacie  [Encoded Polyline Algorithm](https://developers.google.com/maps/documentation/utilities/polylinealgorithm). W bibliotece zaimplementowane są [odpowiednie funkcje](https://github.com/pszufe/OpenStreetMapX.jl/blob/master/src/polyline.jl), które pozwalają na kodowanie/dekodowanie współrzędnych na ten format:

In [107]:
OpenStreetMapX.decode("_p~iF~ps|U_ulLnnqC_mqNvxq`@")

3-element Array{Tuple{Float64,Float64},1}:
 (38.5, -120.2)    
 (40.7, -120.95)   
 (43.252, -126.453)